<a href="https://colab.research.google.com/github/King-prime-x/deep-learning/blob/main/FIneTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Mental-Health-Twitter.csv.zip to Mental-Health-Twitter.csv.zip


In [ ]:
import zipfile
import os

zip_file = "Mental-Health-Twitter.csv.zip"
extract_folder = "unzipped_folder"  # Change if needed

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"Extracted files to: {extract_folder}")


Extracted files to: unzipped_folder


In [ ]:
import pandas as pd

dataset_path = os.path.join(extract_folder,"Mental-Health-Twitter.csv")
dataset = pd.read_csv(dataset_path)
print(dataset.head())
print(dataset.describe())

   Unnamed: 0             post_id                    post_created  \
0           0  637894677824413696  Sun Aug 30 07:48:37 +0000 2015   
1           1  637890384576778240  Sun Aug 30 07:31:33 +0000 2015   
2           2  637749345908051968  Sat Aug 29 22:11:07 +0000 2015   
3           3  637696421077123073  Sat Aug 29 18:40:49 +0000 2015   
4           4  637696327485366272  Sat Aug 29 18:40:26 +0000 2015   

                                           post_text     user_id  followers  \
0  It's just over 2 years since I was diagnosed w...  1013187241         84   
1  It's Sunday, I need a break, so I'm planning t...  1013187241         84   
2  Awake but tired. I need to sleep but my brain ...  1013187241         84   
3  RT @SewHQ: #Retro bears make perfect gifts and...  1013187241         84   
4  It’s hard to say whether packing lists are mak...  1013187241         84   

   friends  favourites  statuses  retweets  label  
0      211         251       837         0      1  
1     

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
df = load_dataset(extract_folder)
df['train'] = df['train'].shuffle(seed = 42)
print(df['train'][:5])

In [ ]:
df = df.remove_columns(['Unnamed: 0','post_id', 'post_created', 'user_id', 'followers', 'friends', 'favourites', 'statuses', 'retweets'])
print(df.column_names)

{'train': ['post_text', 'label']}


In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification

model_path = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path, num_labels = 2)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ['query','value'],
    lora_dropout = 0.1,
    bias = 'none',
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
import torch
device = torch.device('cuda')if torch.cuda.is_available()else torch.device('cpu')
model.to(device)

In [ ]:
def tokenize_function(example):
  return tokenizer(example['post_text'], padding = 'max_length', truncation = True, max_length = 128)

tokenized_data = df.map(tokenize_function, batched = True)

In [ ]:
print(tokenized_data.column_names,'\n',tokenized_data['train'][0])

{'train': ['post_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask']} 
 {'post_text': "🇷🇺@CNNPolitics   YOU'LL KNOW who you are!!!!!! WE KNOW who you are!!!!!!!!  EVERYBODY KNOWS YOU, watches YOU and so… https://t.co/1IdywQMglb", 'label': 0, 'input_ids': [101, 100, 1030, 13229, 18155, 18291, 2015, 2017, 1005, 2222, 2113, 2040, 2017, 2024, 999, 999, 999, 999, 999, 999, 2057, 2113, 2040, 2017, 2024, 999, 999, 999, 999, 999, 999, 999, 999, 7955, 4282, 2017, 1010, 12197, 2017, 1998, 2061, 1529, 16770, 1024, 1013, 1013, 1056, 1012, 2522, 1013, 1015, 3593, 2100, 2860, 4160, 24798, 20850, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
data = tokenized_data['train'].train_test_split(test_size = 0.2, seed = 42)
print(data)
print(data['test'][0])

DatasetDict({
    train: Dataset({
        features: ['post_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['post_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4000
    })
})
{'post_text': 'Love someone in addiction? "I found a paradox, that if you love until it hurts, there can be no more hurt, only more love"- Mother Theresa', 'label': 1, 'input_ids': [101, 2293, 2619, 1999, 13449, 1029, 1000, 1045, 2179, 1037, 20506, 1010, 2008, 2065, 2017, 2293, 2127, 2009, 13403, 1010, 2045, 2064, 2022, 2053, 2062, 3480, 1010, 2069, 2062, 2293, 1000, 1011, 2388, 14781, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
train_data = data['train']
test_data = data['test']
test_data = test_data.train_test_split(test_size = 0.25)
val_data = test_data['train']
test_data = test_data['test']
print(train_data, test_data, val_data)

Dataset({
    features: ['post_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 14400
}) Dataset({
    features: ['post_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4000
}) Dataset({
    features: ['post_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1600
})


In [ ]:
def format_data(examples):
  return examples.remove_columns(['post_text']).rename_column('label','labels').with_format('torch')
train_data = format_data(train_data)
test_data = format_data(test_data)
val_data = format_data(val_data)

In [ ]:
print(train_data[0],test_data[0],val_data[0])

{'labels': tensor(0), 'input_ids': tensor([  101, 11857, 28514,  2758,  4841,  2180,  1005,  1056, 19242,  8037,
        10851,  4259,  2457,  9930, 16770,  1024,  1013,  1013,  1056,  1012,
         2522,  1013,  1017,  2361,  2549,  4160,  2509,  6977, 20492,  2140,
         3081,  1030, 21301, 19894, 18155,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [ ]:
'''from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size = 8, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = 8)
val_dataloader = DataLoader(val_data, batch_size = 8)

for batch in train_dataloader:
  print(batch['labels'])'''

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_metrix
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = argmax(logits, axis = -1)
  accuracy = accuracy_score(labels, predictions)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average = 'weighted')
  roc_auc = roc_auc_score(labels, logits[:,-1])
  confuse = confusion_metrix(labels, predictions)

  return{
      'accuracy' : accuracy,
      'precision' : precision,
      'recall' : recall,
      'f1' : f1,
      'roc_auc' : roc_auc,
      'confusion_metrix' : confuse
  }

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = './results',
    learning_rate = 3e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 3,
    logging_strategy = 'epoch',
    eval_strategy = 'epoch',
    save_startegy = 'epoch',
    warmup_ratio = 0.1,
    load_best_model_at_end = True,
    report_to = 'none'
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

trainer.train()

eval_results = trainer.evaluate()

trainer.save_model('./fine_tuned_bert')